<a href="https://colab.research.google.com/github/lblogan14/reinforcement_learning_with_tensorflow/blob/master/ch4_PG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%cd /content/drive/My' 'Drive/Colab' 'Notebooks/Reinforcement_Learning_with_TensorFlow/

/content/drive/My Drive/Colab Notebooks/Reinforcement_Learning_with_TensorFlow


In [0]:
#installing dependencies
!apt-get -qq -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1 > /dev/null
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!apt-get -qq -y install xvfb freeglut3-dev ffmpeg> /dev/null
!pip3 -q install gym
!pip3 install piglet
!pip3 -q install pyglet
!pip3 -q install pyopengl
!pip3 -q install pyvirtualdisplay
!apt install xvfb -y

E: Unable to locate package libcusparse8.0
E: Couldn't find any package by glob 'libcusparse8.0'
E: Couldn't find any package by regex 'libcusparse8.0'
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'
    100% |████████████████████████████████| 61kB 3.7MB/s 
    100% |████████████████████████████████| 92kB 11.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/c7/59/a5/5bd1a35a4a4596714c4c7925a1751e7b1580b6ced363fd7969
Successfully built piglet
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


#Review:
Value-based approach: (solved by dynamic programming)
* Value iteration
* Policy iteration
* Q-learning

Policy optimization approach:
* Policy gradients
* Actor-critic algorithms

As per the dynamic programming method, there are a set of self-consistent equations to
satisfy the $Q$ and $V$ values. Policy optimization is different, where policy learning happens
directly. Remind that the value function can be derived from the following diagram:
![alt text](https://github.com/lblogan14/reinforcement_learning_with_tensorflow/blob/master/note_images/ch4/value_func.PNG?raw=true)

Value-based methods learn the value function and an implicit policy can be derived, whereas no value function with policy-based methods can be learned and the policy is learnt directly. 

The actor-critic method is to learn both the value function and the policy. The network which learns the value function acts as a critic to the policy network which is the actor.

#Policy Optimization Method
The objective of the policy optimization method is to find the stochastic policy $\pi_{\theta}$ that is a distribution of actions for a given state that maximizes the expected sum of rewards. This method aims to find the optimal policy directly. The neural network can be applied here to take the state information as input and output the distribution of possible acitons.

Policy optimization:
* **weight parameter** of the neural network is the parameter of the control policy, defined by $\theta$ vector. Thus, the optimized $\theta$ is the best policy. The optimal policy has the maximum overall reward. The expected sum of rewards is defined as $\max_{\theta}E[\sum\limits_{t=0}^H \gamma^t R(s)|\pi_{\theta}]$ \\
where $H$ is the time step at horizon. If the start time step $t=0$, then the total time steps are $H+1$
* **stochastic policy** smooths out the policy optimization problems, unlike the deterministic policy in a grid-world environment where changes owing to change in action is not smooth. \\
In a stochastic policy, $\pi_{\theta}(a|s)$ gives the probability of action $a$ for a given state $s$. Thus, $\pi_{\theta}$ gives the probabilistic distribution of actions for a given state.

#Policy Objective Functions
The objective in a policy method is to find the best parameter vector given policy $\pi_{\theta}(s,a)$ with parameter vector $\theta$. To evaluate the quanlity of the policy $\pi_{\theta}(s,a)$ for different values of the parameter vector $\theta$, the objective function $J(\theta)$ is introduced.

There are several methods to measure this quality of a policy $\pi_{\theta}(s,a)$:
* For *episodic environment*, $J(\theta)$ is the value function of the start state $V^{\pi_{\theta}}(s_1)$ if it starts from any state $s_1$. Thus, the *value function* is the *expected sum of reward* from that state onwards,
$$J(\theta)=V^{\pi_{\theta}}(s)$$
* For *continuing environment*, $J(\theta)$ is the average value function of the states. Thus, $J(\theta)$ can be the summation of the probability of being in any state $s$ which is $d^{\pi_{\theta}}(s)$ times the value of that state, the expected reward from that state onward:
$$J(\theta)=\sum\limits_s d^{\pi_{\theta}}(s)V^{\pi_{\theta}}(s)$$
* for *continuing environment*, $J(\theta)$ can be the average reward per time step which is the summation of the probability of being in any state $s$ which is $d^{\pi_{\theta}}(s)$ times the expected reward over different actions for that state, $E[R^a_s]=\sum\limits_a \pi_{\theta}(s,a)R^a_s$:
$$J(\theta)=\sum\limits_s d^{\pi_{\theta}}(s)\sum\limits_a \pi_{\theta}(s,a)R^a_s$$
where, $R^a_s$ is the reward at state $s$ for taking aciton $a$.

The gradient-based optimization method is applied here to get greater efficiency. Since $J(\theta)$ measures the quality of a policy, the gradient ascend method is applied to find the maximum with respect to parameter $\theta$,
$$\theta^*\leftarrow\theta+\alpha\nabla_{\theta}J(\theta)$$
where $\nabla_{\theta}J(\theta)$ is the policy gradient and $\alpha$ is the learning rate, called **step size parameter**.

The policy gradient can be written as:
$$\nabla_{\theta}J(\theta)=\begin{bmatrix}
                                                       \frac{\nabla J(\theta)}{\nabla\theta_1} \\
                                                       \frac{\nabla J(\theta)}{\nabla\theta_2} \\
                                                       ... \\
                                                       \frac{\nabla J(\theta)}{\nabla\theta_n}
                                                 \end{bmatrix}$$

##Policy Gradient Theorem 
Given differentiable non-zero policy $\pi_{\theta}(s,a)$, the gradient of this policy is $\nabla_{\theta}\pi_{\theta}(s,a)$:
$$\nabla_{\theta}\pi_{\theta}(s,a)=\pi_{\theta}(s,a)\frac{\nabla_{\theta}\pi_{\theta}(s,a)}{\pi_{\theta}(s,a)} =_{\theta}\pi_{\theta}(s,a)\nabla_{\theta}\log\pi_{\theta}(s,a)$$
where $\nabla_{\theta}\log\pi_{\theta}(s,a)$ is called the score function.

Consider one-step MDP,
* Starting state is $s$ whose probability of occurring is $d^{\pi_{\theta}}(s)$
* Terminal state happens after one timestep with reward $r=R^a_s$

For a continuing environment, $J(\theta)=\sum\limits_s d^{\pi_{\theta}}(s)\sum\limits_a \pi_{\theta}(s,a)R^a_s$. Thus, the policy gradient is given as:
$$\nabla_{\theta}J(\theta)=\sum\limits_s d^{\pi_{\theta}}(s)\sum\limits_a \nabla_{\theta}\pi_{\theta}(s,a)R^a_s \\
=\sum\limits_s d^{\pi_{\theta}}(s)\sum\limits_a \pi_{\theta}(s,a)\nabla_{\theta}\log\pi_{\theta}(s,a) R^a_s \\
=E_{\pi_{\theta}}[\nabla_{\theta}\log\pi_{\theta}(s,a)r]$$

Similarly, consider multi-step MDP, replace instantaneous reward $r$ by the state-action $Q$ value function $Q^{\pi}(s,a)$. This is the **policy gradient theorem**. Thus, for any such policy objective function and any differentiable $\pi_{\theta}(s,a)$, the policy gradient is given by:
$$\nabla_{\theta}J(\theta)=E_{\pi_{\theta}}[\nabla_{\theta}\log\pi_{\theta}(s,a)Q^{\pi_{\theta}}(s,a)]$$

#Temporal Difference Rule
Definition: **Temporal difference (TD)** is the difference of the value estimates between two time steps. In the temporal difference learning, only one time step ahead is done and a value estimate of the state at the next time step is usd to update the current state's value estimate.

##TD(1) Rule
is combined with the concept of eligibility trace.

Each `Episode T` starts with:
  * eligibility score $e(s)=0$ for all states $s$
  * $V_T(s)=V_{T-1}(s)$, value of the state in given `Episode T` for all states $s$.

Then, at each time step of the episode, update the eligibility of the state $s_{t-1}$:
  * The state value function using temporal difference error for the current leaving
state $s_{t-1}$ which is $r_t+\gamma V_{T-1}(s_t)-V_{T-1}(s_{t-1})$.
$$e(s)$$
  * The eligibility score is discounted with the given discounting factor
$$e(s)=\gamma e(s)$$

All these updates for all states are independent, so they can be performed in parallel. The efficiency of TD(1) method is the same as the Monte Carlo approach.

##TD(0) Rule
finds the value estimate for the any finite data that is repeated infinitely. *Infinitely-repeated finite data* means to take the finite data and keep running the estimate update rule over and over again. The estimate update rule is given by:
$$V_T(s_{t-1})=V_T(s_{t-1})+\alpha_T[r_t+\gamma V_{T-1}(s_t)-V_{T-1}(s_{t-1})]$$
Therefore, the value function is given by:
$$V_T(s_{t-1})=E_{s_{t}}[r_t+\gamma V_{T-1}(s_t)]$$
This is completely different from the outcome-based model, e.g. Monte Carlo, where an estimate of the state $V_T(s_t)$ is not used. TD(0) rule uses the whole sequence of rewards till the end of the episode.

In the outcome-based model of the Monte Carlo approach, the value function is given by:
$$$V_T(s_{t-1})=E[r_t+\gamma r_{t+1}+\gamma^2 r_{t+2}+......]$$

The difference between TD(0) and TD(1) is the use the eligibility trace to update the state-value function in case of TD(1) but not in case of TD(0). The value function of the current leaving state $s_{t-1}$ is only updated using the temporal difference error, $r_t+\gamma V_{T-1}(s_t)-V_{T-1}(s_{t-1})$

##TD($\lambda$) Rule
This is a generalized rule, where $\lambda\in[0,1]$ and
* $\lambda=0$, TD($\lambda$) tends to TD(0)
* $\lambda=1$, TD($\lambda$) tends to TD(1)

The estimate update rules are:
$$V_T(s)=V_T(s)+\alpha_T[r_t+\gamma V_{T-1}(s_t)-V_{T-1}(s_{t-1})]e(s)$$
$$e(s)=\lambda\gamma e(s)$$

#Policy Gradients
Recall: for any differentiable policy $\pi_{\theta}(s,a)$, the policy gradient is given by
$$\nabla_{\theta}J(\theta)=E_{\pi_{\theta}}[\nabla_{\theta}\log\pi_{\theta}(s,a)Q^{\pi_{\theta}}(s,a)]$$

##Monte Carlo Policy Gradient
Parameters are updated by the stochastic gradient ascent method. \\
pseduocode: \\
`intialize` $\theta$ `arbitrarily` \\
`for each episode as per the current policy` $\pi_{\theta}$ `do` \\
$\quad$ `for step t=1 to T-1 do` \\
$\quad\quad \theta\leftarrow\theta+\alpha\nabla_{\theta}\log\pi_{\theta}(s_t,a_t)v_t$ \\
$\quad$ `end for` \\
`end for`
`Output: final` $\theta$

where $v_t$ is an unbiased sample of $Q^{\pi_{\theta}}(s_t,a_t)$ and it is the cumulative reward from that time-step onward.

##Actor-critic algorithms
MCPG leads to high variance. To solve this issue, a critic is used to estimate the state-action value function:
$$Q_w(s,a)\approx Q^{\pi_{\theta}}(s,a)$$
The **actor-critic algorithm** maintains two netowrks:
* The **critic** network updates the weight parameter $w$ of the function approximator of the state-action
* The **actor** network updates the policy parameter $\theta$ as per the direction given by the critic

![alt text](https://github.com/lblogan14/reinforcement_learning_with_tensorflow/blob/master/note_images/ch4/actor-critic.PNG?raw=true)

Therefore, the true state-action value funciton $Q^{\pi_{\theta}}(s,a)$ from the actual policy gradient formula is replaced with an approximate state-action value function $Q_w(s,a)$. Now the policy gradient and the update of policy parameter $\theta$ are replaced with:
$$\nabla_{\theta}J(\theta)\approx E_{\pi_{\theta}}[\nabla_{\theta}\log\pi_{\theta}(s,a)Q_w(s,a)]$$,
$$\Delta\theta=\alpha\nabla_{\theta}\log\pi_{\theta}(s,a)Q_w(s,a)$$

The critic network uses TD(0) rule to update the weight parameters $w$ in order to estimate the state-action value function, and the actor network uses policy gradients to update the policy parameter $\theta$.

The actor-critic algorithm uses both value-based optimzation and policy-based optimization. Unlike the Monte Carlo policy gradient approach where the policy improvement happens greedily. The actor updates the policy parameter in the actor-critic approach taking a step in the direction as per the critic in order to get a better policy.

##Use a baseline to reduce variance
Subtracting a **baseline function** $b(s)$ from the policy gradient to reduce variance will reduce the variance without affecting the expectation value. (Mathematical derivation is not shown)

A good choice of baseline functions would be the state value function
$$b(s)=V^{\pi_{\theta}}(s)$$
Then, the policy gradient can be rewritten as:
$$\nabla_{\theta}J(\theta)=E_{\pi_{\theta}}[\nabla_{\theta}\log\pi_{\theta}(s,a)[Q^{\pi_{\theta}}(s,a)-V^{\pi_{\theta}}(s)]]$$
where $Q^{\pi_{\theta}}(s,a)-V^{\pi_{\theta}}(s)$ is called the **advantage function** $A^{\pi_{\theta}}(s,a)$.
$$\nabla_{\theta}J(\theta)=E_{\pi_{\theta}}[\nabla_{\theta}\log\pi_{\theta}(s,a)A^{\pi_{\theta}}(s,a)]$$
Thus, the expected value is under control by lowered variance without any change in the direction by using a baseline function.

##Vanilla policy gradient
The goal of vanilla policy gradient is to update the policy using the policy gradient estimate with better baseline estimation.

Pseduocode:

`initialize: Policy parameter` $\theta$ `and baseline b` \\
`for iteration = 1, 2,..., N do` \\
$\quad$ `collect a set of trajectories using the current policy` \\
$\quad$ `at each time step t in each trajectory, compute the following:` \\
$\quad$ $\quad$ `returns` $R_t=\sum\limits_{t'=t}^{T-1}\gamma^{t'-t}r_{t'}$ `and` \\
$\quad$ $\quad$ `advantage estimate` $\hat{A}_t=R_t-b(s_t)$ \\
$\quad$ `refit the baseline function` $b(s_t)$ \\
$\quad \delta=\sum\limits_{t=1}^{T-1}\nabla_{\theta}\log\pi(a_t|s_t,\theta)\hat{A_t}$ \\
$\quad \theta=\theta + \alpha\delta$ \\
`end for`

#Agent Learning Pong Using Policy Gradients
Environment: **pong-v0** from OpenAI gym \\
Policy network: single hidden layer neural network fully connected to the raw pixels of pong at the input layer and also to the ouput layer containing a single node returning the probability of the paddle going up. \\
Input image: 80x80 grayscale, not RGB

Neural network:
* **input layer**: 80x80 squashed into 6400x1, which represents 6400 nodes
* **hidden layer**: 200 nodes
* **output layer**: 1 node

Parameters:
* **weights and bias connecting input and hidden layer**: 6400x200 (weights) + 200 (bias)
* **weights and bias connecting hidden and output layer**: 200x1 (weights) + 1 (bias)

Note: \\
The Pong environment cannot be played from a static frame, thus the motion information needs to be captured. Since the convolutional neural network is not used here, no spatial information is available. The motion information can be captured by concatenating two frames or the difference between the new and the previous frame. \\
To implement reinforcement learning, try lots of tasks and note down the observations, and then perform the tasks that performed better:
1. Initialize a random policy
2. For the given current policy, collect different sample trajectories (rollouts) in the following way
  1. Run a single episode of the game and capture the trajectories in that episode.
  2. Collect a batch of trajectories.
  
The cases of winning a game are considered to be the correct label for the action and the cases of losing a game are wrong label. Thus, the product of advantage and log probability of actions $\sum\limits_{i}A_i\times\log p(y_i|x_i)$ are maximized after collecting the batch of trajectories.

 $A_i$ is the advantage associated with a state action pair. Advantage is a *scalar* quantity, which quantifies how good the action eventually turned out. $A_i$ would be high if the given action is encouraged in future and low if the action is disenrouraged. A positive advantage makes the action more likely to occur in future for that state, while a
negative advantage makes the action less likely to occur in future for that state.

##Implementation 

In [0]:
import numpy as np
import _pickle as pickle #same/load model
import gym

###Hyperparameter initialization

In [0]:
H = 200 #number of nodes in the hidden layer
batch_size = 10
learning_rate = 1e-4
gamma = 0.99 # discount factor
decay_rate = 0.99 # for RMPProp Optimizier -> gradient descent
resume = False # to resume from previous checkpoint

###Policy neural network model initialization

In [0]:
D = 80*80 # input dimension
if resume:
  model = pickle.load(open('model.v', 'rb'))
else:
  model = {}
  # Xavier initialization of weights
  model['W1'] = np.random.randn(H,D) * np.sqrt(2.0/D)
  model['W2'] = np.random.randn(H) * sqrt(2.0/H)

# save gradients which can be summed up over a batch
grad_buffer = {k: np.zeros_like(v) for k,v in model.iteritems()}
# save the value of rms prop formula
rmsprop_cache = {k: np.zeros_like(v) for k,v in model.iteritems()}

###Activation functions

In [0]:
def sigmoid(x):
  return 1.0/(1.0 + np.exp(-x))

def relu(x):
  x[x<0] = 0
  return x

###Preprocessing function
takes in the image pixels as the parameter and preprocesses them by *cropping*, *downsampling*, *making* it grayscale, *erasing* the background, and *flattening* the image to a one-dimensional vector.

In [0]:
def prepro(I): # I = single frame of game as the input
  '''preprocess 210x160x3 unit8 frame into 6400x1 (80x80) 1D float vector'''
  I = I[35::195] # cropping only containing paddles and ball
  I = I[::2, ::2, 0] # downsample by factor of 2 and only take R channel
  I[I==144] = 0 # erase background type 1
  I[I==109] = 0 # erase background type 2
  I[I!=0] = 1 # everything else set of 1 except paddles and ball
  return I.astype('float').ravel() # flatten to 1D

###Discount rewards
takes a list of rewards $r$ corresponding to different time-steps as the parameters and returns a list of discounted rewards corresponding to different time-steps

In [0]:
def discount_rewards(r):
  '''take 1D float array of rewards and compute discounted reward'''
  discount_r = np.zeros_like(r)
  running_add = 0 # addition of rewards
  for t in reversed(xrange(0, r.size)):
    if r[t] != 0: # episode ends
      running_add = 0
    running_add = gamma*running_add + r[t]
    discount_r[t] = running_add
  return discount_r

###Forward propagation
takes in the preprocessed image vector and returns the probability of action being `UP` and a vector containing the value of hidden state nodes

In [0]:
def policy_forward(x):
  h = np.dot(model['W1'], x)
  h = relu(h)
  logit = np.dot(model['W2'], h)
  p = sigmoid(logit)
  return p, h
  #probability of action 2(i.e. UP) and hidden layer state (i.e. hidden state)

###Backward propagation
takesin the hidden state values, error `gradient_logp`, and observations to compute the derivatives with respect to different weight parameters.

In [0]:
def policy_backward(arr_hidden_state, gradient_logp, observation_values):
  '''backward pass'''
  # arr_hidden_state is array of intermediate hidden states   shape[200x1]
  # gradient_logp is the loss value   shape[1x1]
  
  dW2 = np.dot(arr_hidden_state.T, gradient_logp).ravel()
  # [200x1].[1x1] => [200x1] => flatten => [1x200]
  
  dh = np.outer(gradient_logp, model['W2'])
  # [1x1]outer[1x200] => [1x200]
  
  dh = relu(dh)
  
  dW1 = np.dot(dh.T, observation_values)
  # [200x1].[1x6400] => [200x6400]
  
  return {'W1':dW1, 'W2':dW2}

###Main implementation and Training

In [0]:
env = gym.make('Pong-v0')
observation = env.reset()
prev_x = None
'''
prev frame value in order to compute the difference between current and previous frame
#as discussed frames are static and the difference is used to capture the motion
#Intially None because there's no previous frame if the current frame is the 1st frame of the game
'''
episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 0

In [0]:
# Training
while True:
  env.render()
  # get the input and preprocess it
  cur_x = prepro(observation)
  # get the frame difference which would be the input to the network
  if prev_x is None:
    prev_x = np.zeros(D)
  x = cur_x - prev_x
  prev_x = cur_x
  
  #forward propagation of the policy network
  #sample an action from the returned probability
  aprob, h = policy_forward(x)
  #stochastic process
  if np.random.uniform() < aprob:
    action = 2
  else:
    action = 3
    
  episode_observation.append(x) # record observation
  episode_hidden_layer_values.append(h) # record hidden state
  if action == 2:
    y = 1
  else:
    y = 0
    
  episode_gradient_log_ps.append(y-aprob) # record the gradient
  
  #new step in the environment
  observation, reward, done, info = env.step(action)
  reward_sum+=reward #for advantage purpose
  episode_rewards.append(reward) # record the reward
  
  if done: # if the episode is over
    episode_number += 1
    
    # stack inputs, hidden_states, actions, gradients_logp, rewards for the episode
    arr_hidden_state = np.vstack(episode_hidden_layer_values)
    gradient_logp = np.vstack(episode_gradient_log_ps)
    observation_values = np.vstack(episode_observations)
    reward_values = np.vstack(episode_rewards)
    
    # reset the memory arrays
    episode_hidden_layer_values, episode_observations, episode_gradient_log_ps, episode_rewards = [],[],[],[]
    
    # discounted reward computation
    discounted_episoderewards = discount_rewards(reward_values)
    # normalize discounted_episoderewards i.e. obtain advantage
    discounted_episoderewards = (discounted_episoderewards - np.mean(discounted_episoderewards)) / np.std(discounted_episoderewards)
    
    # modulate the gradient with the advantage
    gradient_logp *= discounted_episoderewards
    
    grad = policy_backward(arr_hidden_state, gradient_logp, observation_values)
    
    # sum the gradient over the batch size
    for layer in model:
      grad_buffer[layer] += grad[layer]
      
    # perform RMSProp to update weights after every 10 episodes
    if episode_number % batch_size == 0:
      epsilon = 1e-5
      for weight in model.keys():
        g = grad_buffer[weight] # gradient
        rmsprop_cache[weight] = decay_rate*rmsprop_cache[weight] + (1-decay_rate)*g**2
        model[weight] += learning_rate*g / (np.sqrt(rmsprop_cache[weight]) + epsilon)
        grad_buffer[weight] = np.zeros_like(model[weight])
        
    if running_reward is None:
      running_reward = reward_sum
    else:
      running_reward = running_reward*learning_rate+reward_sum*(1-learning_rate)

    print('Episode Reward : {}, Running Mean Award : {}'.format(reward_sum,running_reward))
    if episode_number % 100 == 0:
      pickle.dump(model,open('model.v','wb'))

    reward_sum = 0
    prev_x = None
    observation = env.reset() # reset the environment since episode has ended
    
  if reward != 0: # if reward is either +1 or -1 i.e. an episode has ended
    print('Episode {} ended with reward {}'.format(episode_number, reward))

To accelerate the training process, enable the GPU computing setting. It may still take 5-7 hours to complete the training.